In [1]:
import serial

import numpy as np

import librosa

from keras.models import load_model

import pandas as pd

In [5]:
ser.close()

In [6]:
# 시리얼 포트 설정
ser = serial.Serial('COM5', 9600)  # COMX는 아두이노가 연결된 포트를 나타냅니다.

In [7]:
def extract_features(sample):
    feature_list = []
    for j in range(4):
        sub_sample = sample[j].flatten()
        mel_spectrogram = librosa.feature.melspectrogram(y=sub_sample, sr=49, n_fft=7, n_mels=7)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        mfcc = librosa.feature.mfcc(S=log_mel_spectrogram, sr=49)
        
        feature = np.concatenate((log_mel_spectrogram.flatten(), mfcc.flatten()))
        feature_list.append(feature)
        
    return np.array(feature_list).reshape(1, 14, 4)

In [8]:
model_clf = load_model('siren_clf_model.h5')
model_loc = load_model('siren_loc_model.h5')

In [26]:
df = pd.DataFrame(columns=['x', 'y', 'dt'])
df.to_csv('audio_sensor_output.csv', index=None)

In [27]:
# 센서 온
state = 'mic_sen_on'
while state == 'mic_sen_on':
    
    # 인풋 받기
    input_list = []
    
    ser.readline()

    while len(input_list) < 4*49:
        str_values = ser.readline().decode('utf-8').strip().split(",")
        if len(str_values) < 5:
            input_list = []
            continue
        int_list = [int(str_value) for str_value in str_values[0:4]]
        for str_value in int_list:
            input_list.append(str_value)
        
    input_list = input_list[-49*4:]
    
    
    # 인풋 전처리
    x1 = np.array(input_list).reshape(7,7,4).astype(float).reshape(1, 7, 7, 4)
    x2 = extract_features(np.array(input_list).astype(float)).reshape(1, 1, 14, 4)
     
    print(len(input_list))    

    # 모델 추론
    clf_label = model_clf.predict([x1,x2])
    if clf_label[0][0] >= 0.49294052: #threshold
        loc_label = model_loc.predict([x1,x2])
        x, y = loc_label[0][0], loc_label[0][1]
        
        dt = ser.readline().decode('utf-8').strip().split(",")[4]
        df2 = pd.DataFrame([{'x':x, 'y':y, 'dt': dt}])
        df = pd.concat([df, df2], ignore_index=True)
        df.to_csv('audio_sensor_output.csv', index=None)
        
        print(df.iloc[-1,:])
    ser.readline()

196
1/1 [==============================] - ETA: 0s

C:\Users\chanj\.conda\envs\zolnon\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=7 is too large for input signal of length=1
  warnings.warn(


1/1 [==============================] - 0s 74ms/step
196
1/1 [==============================] - 0s 64ms/step
196
1/1 [==============================] - 0s 63ms/step
196
1/1 [==============================] - 0s 71ms/step
196
1/1 [==============================] - 0s 56ms/step
196
1/1 [==============================] - 0s 82ms/step
196
1/1 [==============================] - 0s 59ms/step
196
1/1 [==============================] - 0s 57ms/step
x      10.59734
y     10.457809
dt         8.48
Name: 0, dtype: object
196


C:\Users\chanj\AppData\Local\Temp\ipykernel_91680\194514340.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df2], ignore_index=True)


1/1 [==============================] - 0s 69ms/step
196
1/1 [==============================] - 0s 127ms/step
x    -0.092467
y     4.140954
dt        8.26
Name: 1, dtype: object
196
1/1 [==============================] - 0s 63ms/step
196
1/1 [==============================] - 0s 59ms/step
196
1/1 [==============================] - 0s 63ms/step
x    -5.074342
y     6.907486
dt       19.13
Name: 2, dtype: object
196
1/1 [==============================] - 0s 65ms/step
196
1/1 [==============================] - 0s 54ms/step
196
1/1 [==============================] - 0s 65ms/step
x    -20.105547
y      4.724832
dt        45.36
Name: 3, dtype: object
196
1/1 [==============================] - 0s 68ms/step
196
1/1 [==============================] - 0s 65ms/step
x    -15.656099
y      7.655375
dt        58.41
Name: 4, dtype: object
196
1/1 [==============================] - 0s 54ms/step
196
1/1 [==============================] - 0s 71ms/step
196
1/1 [==============================] - 0s 58ms/st

...

1/1 [==============================] - 0s 69ms/step
x     -3.192324
y     15.133264
dt        41.74
Name: 315, dtype: object
196
1/1 [==============================] - 0s 64ms/step
x     -2.792209
y     11.926416
dt        41.77
Name: 316, dtype: object
196
1/1 [==============================] - 0s 64ms/step
x    -12.424423
y     11.988211
dt        41.50
Name: 317, dtype: object
196
1/1 [==============================] - 0s 68ms/step
x    -14.355407
y      15.33086
dt        41.74
Name: 318, dtype: object
196
1/1 [==============================] - 0s 58ms/step
x    -11.797382
y      20.33041
dt        31.74
Name: 319, dtype: object
196
1/1 [==============================] - 0s 62ms/step
x     -6.183436
y     13.515128
dt        68.46
Name: 320, dtype: object
196
1/1 [==============================] - 0s 70ms/step
x     -15.84026
y     12.364811
dt        66.40
Name: 321, dtype: object
196
1/1 [==============================] - 0s 60ms/step
x     -5.293892
y     10.817084
dt        67.

1/1 [==============================] - 0s 55ms/step
196
1/1 [==============================] - 0s 55ms/step
196
1/1 [==============================] - 0s 74ms/step
x     10.033355
y      12.69606
dt        10.46
Name: 347, dtype: object
196
1/1 [==============================] - 0s 56ms/step
196
1/1 [==============================] - 0s 71ms/step
196
1/1 [==============================] - 0s 57ms/step
x    -6.775893
y     10.16622
dt       83.81
Name: 348, dtype: object
196
1/1 [==============================] - 0s 60ms/step
x     -0.845965
y     10.094148
dt        63.92
Name: 349, dtype: object
196
1/1 [==============================] - 0s 56ms/step
x    -6.889402
y     5.923903
dt       16.73
Name: 350, dtype: object
196
1/1 [==============================] - 0s 61ms/step
196
1/1 [==============================] - 0s 69ms/step
196
1/1 [==============================] - 0s 54ms/step
196
1/1 [==============================] - 0s 54ms/step
196
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 53ms/step
196
1/1 [==============================] - 0s 65ms/step
x      0.052041
y     15.873295
dt        20.13
Name: 375, dtype: object
196
1/1 [==============================] - 0s 54ms/step
x     12.619855
y      2.615889
dt        19.40
Name: 376, dtype: object
196
1/1 [==============================] - 0s 55ms/step
x    -1.021069
y     5.112644
dt       25.18
Name: 377, dtype: object
196
1/1 [==============================] - 0s 155ms/step
196
1/1 [==============================] - 0s 60ms/step
196
1/1 [==============================] - 0s 59ms/step
196
1/1 [==============================] - 0s 55ms/step
x    -9.402839
y      9.55464
dt       31.93
Name: 378, dtype: object
196
1/1 [==============================] - 0s 58ms/step
196
1/1 [==============================] - 0s 55ms/step
196
1/1 [==============================] - 0s 59ms/step
196
1/1 [==============================] - 0s 57ms/step
196
1/1 [==============================] - 0s

196
1/1 [==============================] - 0s 66ms/step
x    -25.218254
y     13.875379
dt        10.40
Name: 403, dtype: object
196
1/1 [==============================] - 0s 61ms/step
x     34.602913
y      5.347653
dt        10.81
Name: 404, dtype: object
196
1/1 [==============================] - 0s 61ms/step


SerialException: ClearCommError failed (PermissionError(13, '장치가 명령을 인식하지 않습니다.', None, 22))

In [ ]:
ser.close()